In [1]:
import pandas as pd
import numpy as np 
import sklearn
import seaborn as sns
import pandas as pd
from pathlib import Path
import os
import sys
import csv
import numpy as np

import nltk                                                             # Natural language Toolkit
from nltk.stem import SnowballStemmer                                   # Porter's II Stemmer
from nltk import word_tokenize                                          # Document tokenizer
from nltk.corpus import stopwords  
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
df = pd.read_csv('data/fairy_tales_en.csv')

In [3]:
stopwords = nltk.corpus.stopwords.words('english')
remove_pronouns=['he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself']
stopwords = [word for word in remove_pronouns if word not in stopwords]

In [4]:
import string 
import re
#1) Preprocess
### Cleaning Function 
def cleaning(text):
    # Remove Punctuation 
    clean_text=text.translate(str.maketrans('', '', string.punctuation))
    # Remove Symbols 
    clean_text=re.sub(r'[^\w]', ' ', clean_text)
    # Remove Numbers
    #clean_text=clean_text.translate(str.maketrans('', '', string.digits))
    # Put in Lowercase 
    #clean_text=clean_text.lower()
    # Remove StopWords
    var=clean_text.split()
    filtered_list=[]
    for word in var:
        if word not in stopwords:
            filtered_list.append(word)
    clean_text=" ".join(map(str, filtered_list))
    return clean_text


#def stemming(text):
 #   stemming_output = ' '.join([englishStemmer.stem(w) for w in text.split()])
  #return stemming_output 

#load stopwords
df['clean_text']=[cleaning(i) for i in df['text']]

In [39]:
df['clean_text'][200000]

'That night the warriors from the land of the Goths were feasted in the'

In [5]:
df["book_raw_text"]=df.groupby(['gutenberg_id'])['clean_text'].transform(lambda x: ' '.join(x))
df['book_raw_text']

0         THE HEROES OR GREEK FAIRY TALES FOR MY CHILDRE...
1         THE HEROES OR GREEK FAIRY TALES FOR MY CHILDRE...
2         THE HEROES OR GREEK FAIRY TALES FOR MY CHILDRE...
3         THE HEROES OR GREEK FAIRY TALES FOR MY CHILDRE...
4         THE HEROES OR GREEK FAIRY TALES FOR MY CHILDRE...
                                ...                        
242821    SLAV TALES Illustration Illustration From The ...
242822    SLAV TALES Illustration Illustration From The ...
242823    SLAV TALES Illustration Illustration From The ...
242824    SLAV TALES Illustration Illustration From The ...
242825    SLAV TALES Illustration Illustration From The ...
Name: book_raw_text, Length: 242826, dtype: object

In [6]:
df.drop_duplicates(subset = ['gutenberg_id'],inplace = True)
df.reset_index(inplace = True)
df["book_text2"]=[re.sub(r'[A-Z]+(?![a-z])', '', x) for x in df["book_raw_text"]]
def tokenization(text):
    tokens = re.split(' ',text)
    return tokens

df['text_tokenized']= df['book_text2'].apply(lambda x: tokenization(x))

for i in range(len(df)):
    df["text_tokenized"][i]=list(filter(None, df["text_tokenized"][i]))
    df["text_tokenized"][i]=[x.lower() for x in df["text_tokenized"][i]]
    
## lemmatize
wnl = nltk.stem.WordNetLemmatizer()

def lemmatize(s):

    s = [wnl.lemmatize(word) for word in s]
    return s

df = df.assign(text_lemma = df.text_tokenized.apply(lambda x: lemmatize(x)))

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20232\1507606882.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_tokenized"][i]=list(filter(None, df["text_tokenized"][i]))
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20232\1507606882.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_tokenized"][i]=[x.lower() for x in df["text_tokenized"][i]]


### Playing around with Word2Vec

Just training a simple Word2Vec model on the given data. 
Given the minimal preprocessing and the small size of training data,
this is just for visualisation purposes.

1. Use pretrained Word2Vec (differences?)
2. Train on larger corpus (data is loading)

In [7]:
import gensim
female = gensim.models.Word2Vec(df['text_lemma'], min_count=10, vector_size=100)
female.wv.most_similar(positive=['she', 'woman'], topn=30)

[('witch', 0.8128074407577515),
 ('farmer', 0.812404453754425),
 ('boy', 0.7983742952346802),
 ('girl', 0.7864857316017151),
 ('princess', 0.7820737957954407),
 ('husband', 0.7642733454704285),
 ('man', 0.7634108066558838),
 ('lad', 0.7586875557899475),
 ('prince', 0.7575526237487793),
 ('fox', 0.7559700012207031),
 ('lady', 0.7437622547149658),
 ('herself', 0.7404909729957581),
 ('bear', 0.738444447517395),
 ('ogre', 0.7384242415428162),
 ('angry', 0.7374202013015747),
 ('shedragon', 0.736622154712677),
 ('quail', 0.7299108505249023),
 ('sparrow', 0.7228524088859558),
 ('tiger', 0.7227628827095032),
 ('priest', 0.7213254570960999),
 ('widow', 0.71696537733078),
 ('duckling', 0.7157494425773621),
 ('nurse', 0.7153459787368774),
 ('poor', 0.7146952748298645),
 ('loki', 0.7144445776939392),
 ('voice', 0.7085778117179871),
 ('magician', 0.7072442173957825),
 ('maiden', 0.7040152549743652),
 ('rabbit', 0.7038483619689941),
 ('mother', 0.7016187906265259)]

In [60]:
female.wv.most_similar(positive=['woman'], negative = ['man'], topn = 30)

[('chattering', 0.41853976249694824),
 ('little', 0.41317519545555115),
 ('two', 0.41122376918792725),
 ('nouronnihar', 0.39704108238220215),
 ('accompanying', 0.38285401463508606),
 ('together', 0.3704138398170471),
 ('told', 0.36129969358444214),
 ('wooed', 0.3611011505126953),
 ('retainer', 0.3553624749183655),
 ('entertained', 0.35201704502105713),
 ('halfdan', 0.3470708429813385),
 ('bridled', 0.34634095430374146),
 ('men', 0.34315475821495056),
 ('kitchenmaid', 0.34312212467193604),
 ('quarreling', 0.3395988345146179),
 ('betrothed', 0.3393055200576782),
 ('frail', 0.33500659465789795),
 ('destroys', 0.33394140005111694),
 ('papoose', 0.3339385986328125),
 ('tang', 0.3303912580013275),
 ('helper', 0.32840102910995483),
 ('lured', 0.3273230791091919),
 ('moore', 0.324090838432312),
 ('ivy', 0.32304617762565613),
 ('called', 0.32278624176979065),
 ('charmed', 0.32214006781578064),
 ('gudarz', 0.317789763212204),
 ('wearily', 0.3135395348072052),
 ('greyhound', 0.31257039308547974),

In [9]:
female.wv.most_similar(positive=['man'], negative = ['woman'], topn = 30)

[('desirable', 0.3882335126399994),
 ('gad', 0.3858051896095276),
 ('pronounce', 0.37855178117752075),
 ('he', 0.35565584897994995),
 ('undertook', 0.33176374435424805),
 ('hint', 0.32833826541900635),
 ('any', 0.3263907730579376),
 ('himself', 0.3229527175426483),
 ('confirm', 0.3220430314540863),
 ('rt', 0.31863224506378174),
 ('drake', 0.3108677566051483),
 ('something', 0.3044278621673584),
 ('prized', 0.3036625385284424),
 ('furnace', 0.3019540309906006),
 ('handsomer', 0.30030983686447144),
 ('anything', 0.298856645822525),
 ('mourn', 0.28809794783592224),
 ('laddie', 0.28802409768104553),
 ('gilbert', 0.28658607602119446),
 ('sooner', 0.2828972637653351),
 ('crusade', 0.2780119478702545),
 ('communicated', 0.27569907903671265),
 ('modestly', 0.2755510210990906),
 ('succeeds', 0.2750774621963501),
 ('purr', 0.2739391624927521),
 ('this', 0.27375197410583496),
 ('it', 0.2693142294883728),
 ('gilliflower', 0.26882362365722656),
 ('puppet', 0.26813894510269165),
 ('better', 0.267725

In [12]:
female.wv.most_similar(positive=['she'], negative = ['he'], topn = 30)

[('her', 0.5400070548057556),
 ('whuppie', 0.35095661878585815),
 ('bragi', 0.34265321493148804),
 ('herself', 0.33557069301605225),
 ('serendib', 0.3293025493621826),
 ('wearily', 0.3260687589645386),
 ('319', 0.31283852458000183),
 ('husband', 0.31151673197746277),
 ('bribed', 0.30957385897636414),
 ('hreidmar', 0.2970789074897766),
 ('moore', 0.2935079336166382),
 ('ruth', 0.28512439131736755),
 ('little', 0.2747052311897278),
 ('spindle', 0.2696342170238495),
 ('un', 0.26040130853652954),
 ('clung', 0.2563196122646332),
 ('macintyre', 0.2553767263889313),
 ('fancying', 0.25477054715156555),
 ('alcinous', 0.2487962543964386),
 ('talus', 0.2468264102935791),
 ('beautiful', 0.2432371824979782),
 ('girl', 0.24278651177883148),
 ('kent', 0.24163919687271118),
 ('hullo', 0.2404334396123886),
 ('hair', 0.23962914943695068),
 ('wunzh', 0.23682096600532532),
 ('forbear', 0.23535561561584473),
 ('stepdaughter', 0.23452749848365784),
 ('upped', 0.23434209823608398),
 ('vegtam', 0.230796292424

If we want to choose a pretrained model, see below

In [72]:
import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
# load model

# model_name = gensim.downloader.load('model_name')

### WEAT association

In [17]:
def get_word_vector(word_list, model):
    vector = []
    for i in word_list: 
        vector.append(model.wv[i])
    return vector

## all other functions from: https://github.com/chadaeun/weat_replication/blob/master/lib/weat.py

def unit_vector(vec):
    """
    Returns unit vector
    """
    return vec / np.linalg.norm(vec)


def cos_sim(v1, v2):
    """
    Returns cosine of the angle between two vectors
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.clip(np.tensordot(v1_u, v2_u, axes=(-1, -1)), -1.0, 1.0)


def weat_association(W, A, B):
    """
    Returns association of the word w in W with the attribute for WEAT score.
    s(w, A, B)
    :param W: target words' vector representations
    :param A: attribute words' vector representations
    :param B: attribute words' vector representations
    :return: (len(W), ) shaped numpy ndarray. each rows represent association of the word w in W
    """
    return np.mean(cos_sim(W, A), axis=-1) - np.mean(cos_sim(W, B), axis=-1)


def weat_score(X, Y, A, B):
    """
    Returns WEAT score
    X, Y, A, B must be (len(words), dim) shaped numpy ndarray
    CAUTION: this function assumes that there's no intersection word between X and Y
    :param X: target words' vector representations
    :param Y: target words' vector representations
    :param A: attribute words' vector representations
    :param B: attribute words' vector representations
    :return: WEAT score
    """

    x_association = weat_association(X, A, B)
    y_association = weat_association(Y, A, B)


    tmp1 = np.mean(x_association, axis=-1) - np.mean(y_association, axis=-1)
    tmp2 = np.std(np.concatenate((x_association, y_association), axis=0))

    return tmp1 / tmp2

A simple implementation, based on [basic domains/attribute words](https://github.com/chadaeun/weat_replication/tree/master/weat)
This is just a placeholder until we find better domains/target words


In [42]:
female_words = ['woman', 'she', 'her', 'herself', 'girl', 'sister', 'daughter', 'mother', 'hers', 'female', 'queen', 'princess', 'wife']
male_words = ['man', 'he', 'him', 'himself', 'boy', 'brother', 'son', 'father', 'his', 'male', 'king', 'prince', 'husband']
print(female.wv.most_similar(positive=female_words, topn = 15))
print(female.wv.most_similar(positive= male_words, topn = 15))

[('husband', 0.9320796132087708), ('sparrow', 0.9143039584159851), ('farmer', 0.9057610630989075), ('bride', 0.9019930362701416), ('priest', 0.8953858613967896), ('boy', 0.8903483152389526), ('lady', 0.8899587988853455), ('lad', 0.889054000377655), ('nurse', 0.874434769153595), ('horn', 0.8740369081497192), ('granny', 0.8735975027084351), ('servant', 0.8701795339584351), ('magician', 0.8678514361381531), ('weep', 0.8665828108787537), ('maiden', 0.8610840439796448)]
[('farmer', 0.918499767780304), ('princess', 0.9104329943656921), ('wife', 0.9100356101989746), ('priest', 0.9009794592857361), ('servant', 0.8931905627250671), ('horn', 0.8931836485862732), ('emperor', 0.8854930996894836), ('lad', 0.8845152854919434), ('sparrow', 0.8836773037910461), ('bride', 0.882102370262146), ('magician', 0.8811991810798645), ('message', 0.8771030306816101), ('mother', 0.8769028186798096), ('ogre', 0.8744143843650818), ('master', 0.8676860928535461)]


In [57]:
pleasant_words = ['caress', 'freedom', 'health', 'love', 'peace', 'cheer', 'friend', 'heaven', 'loyal', 'pleasure',
                   'diamond', 'gentle', 'honest', 'lucky', 'rainbow', 'diploma', 'gift', 'honor', 'miracle', 'sunrise', 'family',
                   'happy', 'laughter', 'paradise', 'vacation']

unpleasant_words = ['abuse', 'crash', 'filth', 'murder', 'sickness', 'accident', 'death', 'grief', 'poison', 'stink',
                    'assault', 'disaster', 'hatred', 'pollute', 'tragedy', 'divorce', 'jail', 'poverty', 'ugly', 'cancer',
                    'kill', 'rotten', 'vomit', 'agony', 'prison']


def get_word_vectors(words, model):
        """
        Returns word vectors represent words
        :param words: iterable of words
        :return: (len(words), dim) shaped numpy ndarrary which is word vectors
        """
        words = [w for w in words if w in model.wv.key_to_index]
        return model.wv[words]

In [64]:
X, Y, A, B = [get_word_vectors(i, female) for i in [female_words, male_words, pleasant_words, unpleasant_words]]

In [65]:
weat_score(X, Y, A, B)

0.3483745

That's a bit surprising but the interpretation is goes analogous to [that](https://developers.googleblog.com/2018/04/text-embedding-models-contain-bias.html): 
> For example, the target lists for the first WEAT test are types of flowers and insects, and the attributes are pleasant words (e.g., "love", "peace") and unpleasant words (e.g., "hatred," "ugly"). The overall test score is the degree to which flowers are more associated with the pleasant words, relative to insects. A high positive score (the score can range between 2.0 and -2.0) means that flowers are more associated with pleasant words, and a high negative score means that insects are more associated with pleasant words.




In [71]:
pip install responsibly

# if anyone is able to pip install that 
# my pip is fucked

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Max retries exceeded with url: /packages/44/64/72211de680c21fe6cea67da182db965861603d311c67839ab39cc7226780/responsibly-0.1.2-py3-none-any.whl (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1129)')))

